In [ ]:
#| default_exp src

In [ ]:
%load_ext autoreload
%autoreload 2

# source functions

In [ ]:
#| export
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support,  accuracy_score
from langchain.pydantic_v1 import BaseModel, Field

In [ ]:
#| export
class UtteranceClassification(BaseModel):
    """Classifying the utterances according to linguistic criteria."""
    gpt_transitivity: str = Field(..., description="Whether this use of the verb should be labelled either `transitive` or `intransitive`.")
    gpt_causativity: str = Field(..., description="Whether this use of the verb should be labelled as `causative` or `anticausative`.")
    gpt_subject_animacy: str = Field(..., description='Whether the subject of the verb should be labelled as `animate` or `inamimate`.')
    gpt_subject_role: str = Field(..., description='Whether the semantic role of the subject of the verb should be labelled as either `agent` or `patient`.')
    gpt_subject: str = Field(..., description='The subject in this utterance that you considered for your classification.')
    gpt_verb: str = Field(..., description='The verb in this utterance that you considered for your classification.')
    gpt_object: str = Field(..., description='The object in this utterance that you considered for your classification.')

In [ ]:
#| export
def get_metrics(results, variable, pos_label):
	gpt_variable = f'gpt_{variable}'
	precision, recall, f1, support = precision_recall_fscore_support(
		results[variable], results[gpt_variable], 
		pos_label=pos_label,
		average='binary',
	)
	metrics = pd.DataFrame(
		columns = ['variable', 'metric', 'score'],
		data = [
			[variable, 'precision', round(precision, 2)],
			[variable, 'recall', round(recall, 2)],
			[variable, 'accuracy', accuracy_score(results[variable], results[gpt_variable])],
			[variable, 'F1', round(f1, 2)]
		])
	return metrics